# Recreating the graphs from the paper and more.
## Experiments
### Apply VIO outlier rejection on the raw data

In [1]:
import os
from Code.UtilityCode.Measurement import Measurement

if not os.path.exists("Data/Measurements_correction"): 
    os.makedirs("Data/Measurements_correction")

for i in range(1, 6):
    sampled_pkl = "Data/Measurements/exp" + str(i) + "_los_sampled.pkl"
    measurement = Measurement()
    measurement.load_sampled_data(sampled_pkl)
    measurement.tb2.vio_frame.outlier_rejection(max_a=0.5)
    measurement.tb3.vio_frame.outlier_rejection(max_a=0.5)
    measurement.tb2.vio_frame.sampled_v = measurement.tb2.vio_frame.v_cor
    measurement.tb3.vio_frame.sampled_v = measurement.tb3.vio_frame.v_cor
    measurement.name = "exp" + str(i) + "_los"
    measurement.save_folder = "Data/Measurements_correction/"
    measurement.save_sampled_data()

### Running the algorihtms
The code bellow runs the analysis. 

The methods have limited parameters accesible in their name. 
For all methods the frequency can be changed. 
For the UPF methods the resample_factor and the sigma_uwb_factor can be changed. By setting multi_particles=0 the UPF is set to a single particle with known initial RP. 
For the Algebraic, QCPQ and NLS method the horizon can be set as well. The horizon reperesent the number of measruments included it time horizon. 
Notice by setting the NLS to a frequency of 1 and an horizon of 10 is equivalent to setting the frequency to 10 with an horizon of 100 and a sampling factor of 10. 

This might take some time to run all methods for all experiments, dependable on your hardware. 
As a reference on a intel CORE vPRO i7 it takes almost 6 hours. Therefor the results are saved in the `results_folder`. 

In [ ]:
import os
from Code.UtilityCode.Measurement import create_experiment, create_experimental_data

data_folder = "Data/Measurements_correction"
results_folder = "Results/experiments"

if not os.path.exists(results_folder): 
    os.makedirs(results_folder)
      
sig_v = 0.08
sig_w = 0.12
sig_uwb = 0.25

experiment_data, measurements = create_experimental_data(data_folder, sig_v, sig_w, sig_uwb)

methods = [
            "losupf|frequency=10.0|resample_factor=0.1|sigma_uwb_factor=1.0",
            "losupf|frequency=10.0|resample_factor=0.1|sigma_uwb_factor=1.0|multi_particles=0",
            "nodriftupf|frequency=10.0|resample_factor=0.1|sigma_uwb_factor=1.0",
            "algebraic|frequency=10.0|horizon=100",
            "QCQP|frequency=10.0|horizon=100",
            "NLS|frequency=1.0|horizon=10",
         ]
tas = create_experiment(results_folder, sig_v, sig_w, sig_uwb)
tas.set_ukf_properties(kappa=-1., alpha=1., beta=2., n_azimuth=4, n_altitude=3, n_heading=4)
tas.debug_bool = True
tas.plot_bool = False
tas.run_experiment(methods=methods, redo_bool=False, experiment_data=experiment_data)


### Load and configure the data

In [ ]:
from Code.Analysis import TwoAgentAnalysis as TAA
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
import seaborn as sns

result_folders = [
                "Results/experiments", 
                ]

variables = ["error_x_relative", "error_h_relative"]
sigma_dv = [0.08]
sigma_uwb = [0.25]

taa = TAA.TwoAgentAnalysis(result_folders=result_folders)

upf_exp = {"Method": "losupf|frequency=10.0|resample_factor=0.1|sigma_uwb_factor=1.0",
           "Variables": {
               "Type": ["experiment"],
               "Variable": variables,
               "Sigma_dv": sigma_dv,
               "Sigma_uwb": sigma_uwb,
               "Frequency": [10.0],
           },
           "Color": "tab:green",
           "Legend": "Ours",
           }
upf_exp_per = {"Method": "losupf|frequency=10.0|resample_factor=0.1|sigma_uwb_factor=1.0|multi_particles=0",
           "Variables": {
               "Type": ["experiment"],
               "Variable": variables,
               "Sigma_dv": sigma_dv,
               "Sigma_uwb": sigma_uwb,
               "Frequency": [10.0],
           },
           "Color": "tab:brown",
           "Legend": "Ours, with known initial RP",
           }
nodriftupf_exp = {"Method": "nodriftupf|frequency=10.0|resample_factor=0.1|sigma_uwb_factor=1.0",
                  "Variables": {
                      "Type": ["experiment"],
                      "Variable": variables,
                      "Sigma_dv": sigma_dv,
                      "Sigma_uwb": sigma_uwb,
                      "Frequency": [10.0],
                  },
                  "Color": "tab:red",
                  "Legend": r"Ours, $\tilde{\text{w}}$ pseudo-state",
                  }
alg_exp = {"Method": "algebraic|frequency=10.0|horizon=100",
           "Variables": {
               "Type": ["experiment"],
              "Variable": variables,
              "Sigma_dv": sigma_dv,
              "Sigma_uwb": sigma_uwb,
               "Frequency": [10.0],
           },
           "Color": "tab:orange",
           "Legend": "Algebraic",
           }
qcqp_exp = {"Method": "QCQP|frequency=10.0|horizon=100",
            "Variables": {
                "Type": ["experiment"],
                "Variable": variables,
                "Sigma_dv": sigma_dv,
                "Sigma_uwb": sigma_uwb,
                "Frequency": [10.0],
            },
            "Color": "tab:blue",
            "Legend": "QCQP",
            }
nls_exp = {
            "Method": "NLS|frequency=1.0|horizon=10",
           "Variables": {
               "Type": ["experiment"],
                "Variable": variables,
                "Sigma_dv": sigma_dv,
                "Sigma_uwb": sigma_uwb,
               "Frequency": [1.0],
           },
           "Color": "tab:purple",
           "Legend": "NLS",
           }


methods_order = [ upf_exp, 
                  upf_exp_per,
                  nodriftupf_exp,
                  alg_exp,
                  qcqp_exp, 
                  nls_exp, 
                ]

df, methods_names, methods_colors, methods_styles, methods_legends = taa.filter_methods_new(methods_order)

### Plotting the boxplots and statistical results
Make sure to have run 

In [18]:
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
import seaborn as sns

taa.print_statistics(methods_names, ["error_x_relative", "error_h_relative"], df)
g = taa.boxplot_exp(df, methods_color=methods_colors, methods_legend=methods_legends,
                hue_variable="Name", hue_order=methods_names,
                col_variable="Variable",
                row_variable=None,
                x_variable="Sigma_dv",
                )

g.axes_dict["error_x_relative"].set_yscale("log")
g.axes_dict["error_h_relative"].set_ylabel(taa.y_label["error_h_relative"])
g.axes_dict["error_x_relative"].set_ylabel(taa.y_label["error_x_relative"])
sns.move_legend(g, loc="upper center", bbox_to_anchor= (0.5, 0.98), ncol=5)
plt.subplots_adjust(top=0.8, bottom=0.12, left=0.06, right=0.99)
# plt.suptitle("Experiments")
plt.show()

losupf|frequency=10.0|resample_factor=0.1|sigma_uwb_factor=1.0|Type_['experiment']|Variable_['error_x_relative', 'error_h_relative']|Sigma_dv_[0.08]|Sigma_uwb_[0.25]|Frequency_[10.0] error_x_relative
2.4896968830626793  pm  1.666813545593958 ; median:  2.0221151530977206
losupf|frequency=10.0|resample_factor=0.1|sigma_uwb_factor=1.0|multi_particles=0|Type_['experiment']|Variable_['error_x_relative', 'error_h_relative']|Sigma_dv_[0.08]|Sigma_uwb_[0.25]|Frequency_[10.0] error_x_relative
1.062527052622213  pm  0.8575427998339417 ; median:  0.8878902287749242
nodriftupf|frequency=10.0|resample_factor=0.1|sigma_uwb_factor=1.0|Type_['experiment']|Variable_['error_x_relative', 'error_h_relative']|Sigma_dv_[0.08]|Sigma_uwb_[0.25]|Frequency_[10.0] error_x_relative
2.758437796676155  pm  1.7020331477310469 ; median:  2.291875813270968
algebraic|frequency=10.0|horizon=100|Type_['experiment']|Variable_['error_x_relative', 'error_h_relative']|Sigma_dv_[0.08]|Sigma_uwb_[0.25]|Frequency_[10.0] error_

KeyboardInterrupt: 

### Time line plots

In [24]:
#reshuffle the order so the lines are visible.
methods_names = [
                "algebraic|frequency=10.0|horizon=100|Type_['experiment']|Variable_['error_x_relative', 'error_h_relative']|Sigma_dv_[0.08]|Sigma_uwb_[0.25]|Frequency_[10.0]", 
                 "QCQP|frequency=10.0|horizon=100|Type_['experiment']|Variable_['error_x_relative', 'error_h_relative']|Sigma_dv_[0.08]|Sigma_uwb_[0.25]|Frequency_[10.0]",
                 "losupf|frequency=10.0|resample_factor=0.1|sigma_uwb_factor=1.0|multi_particles=0|Type_['experiment']|Variable_['error_x_relative', 'error_h_relative']|Sigma_dv_[0.08]|Sigma_uwb_[0.25]|Frequency_[10.0]", 
                 "nodriftupf|frequency=10.0|resample_factor=0.1|sigma_uwb_factor=1.0|Type_['experiment']|Variable_['error_x_relative', 'error_h_relative']|Sigma_dv_[0.08]|Sigma_uwb_[0.25]|Frequency_[10.0]", 
                 "NLS|frequency=1.0|horizon=10|Type_['experiment']|Variable_['error_x_relative', 'error_h_relative']|Sigma_dv_[0.08]|Sigma_uwb_[0.25]|Frequency_[1.0]",
                "losupf|frequency=10.0|resample_factor=0.1|sigma_uwb_factor=1.0|Type_['experiment']|Variable_['error_x_relative', 'error_h_relative']|Sigma_dv_[0.08]|Sigma_uwb_[0.25]|Frequency_[10.0]",
            ]
        
g = taa.lineplot(df, methods_names, methods_colors, methods_styles=methods_styles,
                         methods_legends=methods_legends)
plt.show()

For Method: algebraic|frequency=10.0|horizon=100|Type_['experiment']|Variable_['error_x_relative', 'error_h_relative']|Sigma_dv_[0.08]|Sigma_uwb_[0.25]|Frequency_[10.0] error_x_relative Average over all conditions at each time point: [4.418190797059445, 4.42164264817616, 4.426046809937542, 4.429424812706182, 4.432394453563822, 4.438449454107356, 4.443269249716051, 4.44929693932925, 4.457781413256709, 4.465840117482341, 13.930274198364614, 31.553847073869974, 259.3718263115742, 74862.31442099334, 286.656479339678, 290.65383642046174, 19.97707840895129, 21.821504528572955, 15.96894324450686, 17.170294755537157, 21.80248242140389, 21.813648543614423, 22.133699033023, 18.96747111219114, 14.10013107810239, 14.774136448633858, 18.69171230929168, 18.567485591670863, 20.612210621251904, 22.340516901980717, 20.213966292391582, 22.446437027749656, 21.730404056084126, 21.12735114900112, 21.10137834592686, 21.449950147269035, 21.270856722944917, 20.931641637382594, 20.925159398360087, 20.919192412

## Sim2Real

In [ ]:
import os
from Code.UtilityCode.Measurement import create_experiment, create_experimental_sim_data

data_folder = "Data/Measurements_correction"
results_folder = "Results/sim2real"

if not os.path.exists(results_folder): 
    os.makedirs(results_folder)

sig_v = 0.08
sig_w = 0.12
sig_uwb = 0.25

experiment_data, measurements = create_experimental_sim_data(data_folder, sig_v, sig_w, sig_uwb)
methods = [
            "losupf|frequency=10.0|resample_factor=0.1|sigma_uwb_factor=1.0",
            "losupf|frequency=10.0|resample_factor=0.1|sigma_uwb_factor=1.0|multi_particles=0",
            "nodriftupf|frequency=10.0|resample_factor=0.1|sigma_uwb_factor=1.0",
            "algebraic|frequency=10.0|horizon=100",
            "QCQP|frequency=10.0|horizon=100",
            "NLS|frequency=1.0|horizon=10",
           ]

tas = create_experiment(results_folder, sig_v, sig_w, sig_uwb)
tas.debug_bool = True
tas.plot_bool = False
tas.run_experiment(methods=methods, redo_bool=True, experiment_data=experiment_data, res_type="simulation", prefix="sim_")

['exp3_los_sampled.pkl', 'exp4_los_sampled.pkl', 'exp1_los_sampled.pkl', 'exp2_los_sampled.pkl', 'exp5_los_sampled.pkl']
2024-09-13 13:23:42.625340   losupf|frequency=10.0|resample_factor=0.1|sigma_uwb_factor=1.0 of  ['losupf|frequency=10.0|resample_factor=0.1|sigma_uwb_factor=1.0', 'losupf|frequency=10.0|resample_factor=0.1|sigma_uwb_factor=1.0|multi_particles=0', 'nodriftupf|frequency=10.0|resample_factor=0.1|sigma_uwb_factor=1.0', 'algebraic|frequency=10.0|horizon=100', 'QCQP|frequency=10.0|horizon=100', 'NLS|frequency=1.0|horizon=10']
2024-09-13 13:23:44.184777  Experiment step:  0  / 2200
UPF time 1:  0.2332136631011963
UPF time 0:  0.2372734546661377
2024-09-13 13:23:44.660409  Experiment step:  1  / 2200
UPF time 1:  0.06311321258544922
UPF time 0:  0.061599016189575195
2024-09-13 13:23:44.791527  Experiment step:  2  / 2200
UPF time 1:  0.062059879302978516
UPF time 0:  0.065643310546875
2024-09-13 13:23:44.924102  Experiment step:  3  / 2200
UPF time 1:  0.062406301498413086
U

### Boxplots for comparison with experiments.

In [ ]:
from Code.Analysis import TwoAgentAnalysis as TAA
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
import seaborn as sns

result_folders = [
                "Results/sim2real", 
                ]

variables = ["error_x_relative", "error_h_relative"]
sigma_dv = [0.08]
sigma_uwb = [0.25]

taa = TAA.TwoAgentAnalysis(result_folders=result_folders)


methods_order = [ upf_exp, 
                  upf_exp_per,
                  nodriftupf_exp,
                  alg_exp,
                  qcqp_exp, 
                  nls_exp, 
                ]

df, methods_names, methods_colors, methods_styles, methods_legends = taa.filter_methods_new(methods_order)

## Simulations